### Chapter 4. 스파크 SQL과 데이터 프레임: 내장 데이터 소스 소개
#### 작성자: 이현준(6기 러너)

### Spark Session 생성

In [ ]:
from pyspark.sql import SparkSession
spark = (SparkSession
      .builder
      .appName("Chatper4")
      .getOrCreate())

### spark.sql 예제 88p

In [ ]:
# 데이터 경로
csv_file = "PySpark/data/databricks-datasets/learning-spark-v2/flights/departuredelays.csv"

# 읽고 임시뷰를 생성
# 스키마 추론(더 큰 파일일 경우에는 스키마를 지정해주자)
df = (spark.read.format("csv")
      .option("inferSchema", "true")
      .option("header", "true")
      .load(csv_file))

df.createOrReplaceTempView("us_delay_flights_tbl")

### 비행거리가 1,000마일 이상인 모든 항공편 조회 90p

In [ ]:
spark.sql("""SELECT distance, origin, destination
    FROM us_delay_flights_tbl WHERE distance > 1000
    ORDER BY distance DESC""").show(10)

### 샌프란시스코(SFO)와 시카고(ORD) 간 2시간 이상 지연이 있었던 모든 항공편 조회

In [ ]:
spark.sql("""SELECT date, delay, origin, destination
    FROM us_delay_flights_tbl
    WHERE delay > 120 AND ORIGIN = 'SFO' AND DESTINATION = 'ORD'
    ORDER by delay DESC""").show(10)

### 모든 미국 항공편에 대해 지연에 대한 표시를 레이블로 지정 91p

In [ ]:
spark.sql("""SELECT delay, origin, destination,
              CASE
                  WHEN delay > 360 THEN 'Very Long Delays'
                  WHEN delay > 120 AND delay < 360 THEN 'Long Delays'
                  WHEN delay > 60 AND delay < 120 THEN 'Short Delays'
                  WHEN delay > 0 and delay < 60  THEN  'Tolerable Delays'
                  WHEN delay = 0 THEN 'No Delays'
                  ELSE 'Early'
               END AS Flight_Delays
               FROM us_delay_flights_tbl
               ORDER BY origin, delay DESC""").show(10)

### 파이썬 데이터 프레임 예제 
### 비행거리가 1,000마일 이상인 모든 항공편 조회 92p

In [ ]:
from pyspark.sql.functions import col, desc
(df.select("distance", "origin", "destination")
 .where(col("distance") > 1000)
 .orderBy(desc("distance"))).show(10)

### 샌프란시스코(SFO)와 시카고(ORD) 간 2시간 이상 지연이 있었던 모든 항공편 des

In [ ]:
from pyspark.sql.functions import col, desc
(df.select("date", "delay", "origin", "destination")
 .where((col("delay") > 120) & (col("origin") == 'SFO') & (col("destination") == "ORD"))
 .orderBy(desc("delay"))).show(10)

### 모든 미국 항공편에 대해 지연에 대한 표시를 레이블로 지정

In [ ]:
df.select("date", "delay", "destination",
          when(df.delay > 360, "Very Long Delays")
          .when((df.delay >= 120) & (df.delay <= 360), "Long Delays")
          .when((df.delay >= 60) & (df.delay < 120), "Short Delays")
          .when((df.delay > 0) & (df.delay < 60), "Tolerable Delays")
          .when(df.delay == 0, "No Delays")
          .otherwise("Early").alias("Flight_Delays")
          ).orderBy("origin", df.delay.desc()).show(10)

### SQL 데이터베이스와 테이블 생성하기 93p

### learn_spark_db라는 데이터베이스 생성 및 사용 선언하기

In [ ]:
#// In Scala/Python
spark.sql("CREATE DATABASE learn_spark_db")
spark.sql("USE learn_spark_db")

### 관리형 테이블 생성 94p (기존 코드 오류로 인한 아래 코드로 수정)

In [ ]:
# # // In Scala/Python 기존 코드
# spark.sql("CREATE TABLE managed_us_delay_flights_tbl (date STRING, delay INT, distance INT, origin STRING, destination STRING)")

In [ ]:
# 오류로 인한 수정한 코드
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Chatper4").config("spark.sql.legacy.createHiveTableByDefault", "false").getOrCreate()
#// In Scala/Python
spark.sql("CREATE TABLE managed_us_delay_flights_tbl (date STRING, delay INT,distance INT, origin STRING, destination STRING)")

### 관리형 테이블 생성

In [ ]:
# In Python
# Path to our US flight delays CSV file
csv_file = "PySpark/data/databricks-datasets/learning-spark-v2/flights/departuredelays.csv" # Schema as defined in the preceding example
schema="date STRING, delay INT, distance INT, origin STRING, destination STRING" 
flights_df = spark.read.csv(csv_file, schema=schema) 

# 테이블명이 같은 테이블은 오류를 발생하기 때문에 여기서는 삭제하고 진행
# 위에서 만든 테이블 삭제
spark.sql("DROP TABLE IF EXISTS managed_us_delay_flights_tbl")
flights_df.write.saveAsTable("managed_us_delay_flights_tbl")

### 비관리형 테이블 생성

In [ ]:
# 위에서 만든 테이블 삭제
spark.sql("DROP TABLE IF EXISTS us_delay_flights_tbl")

spark.sql("""CREATE TABLE us_delay_flights_tbl(date STRING, delay INT,distance INT, origin STRING, destination STRING)
    USING csv OPTIONS (
    PATH "PySpark/data/databricks-datasets/learning-spark-v2/flights/departuredelays.csv")"""
    )

In [ ]:
# 덮어쓰기
spark.conf.set("spark.sql.legacy.allowNonEmptyLocationInCTAS", "true")

# 위에서 만든 테이블 삭제
spark.sql("DROP TABLE IF EXISTS us_delay_flights_tbl")

# 데이터 프레임 API에서 사용해야하는 명령어
(flights_df
      .write
      .option("path", "/tmp/data/us_flights_delay")
      .saveAsTable("us_delay_flights_tbl"))

### 임시뷰 예제 95p
### 미국 비행 지연 데이터에서 뉴욕이 출발지인 공항이 있는 하위 데이터셋에 대해서만 작업하는 경우

In [ ]:
# -- In SQL
spark.sql("""CREATE OR REPLACE GLOBAL TEMP VIEW us_origin_airport_SFO_global_tmp_view AS SELECT date, delay, origin, destination from us_delay_flights_tbl WHERE origin = 'SFO'""")
spark.sql("""CREATE OR REPLACE TEMP VIEW us_origin_airport_JFK_tmp_view AS SELECT date, delay, origin, destination from us_delay_flights_tbl WHERE origin = 'JFK'""")

### 96p

In [ ]:
# In Python
df_sfo = spark.sql("SELECT date, delay, origin, destination FROM us_delay_flights_tbl WHERE origin = 'SFO'")
df_jfk = spark.sql("SELECT date, delay, origin, destination FROM us_delay_flights_tbl WHERE origin = 'JFK'")
# Create a 전역 임시 and 임시 뷰
df_sfo.createOrReplaceGlobalTempView("us_origin_airport_SFO_global_tmp_view")
df_jfk.createOrReplaceTempView("us_origin_airport_JFK_tmp_view")

### 전역 임시뷰는 접두사를 사용, 임시뷰는 접두사 없이 접근 가능

In [ ]:
# -- In SQL
spark.sql("""SELECT * FROM global_temp.us_origin_airport_SFO_global_tmp_view""")
spark.sql("""SELECT * FROM us_origin_airport_JFK_tmp_view""")
# // In Scala/Python
spark.read.table("us_origin_airport_JFK_tmp_view")
# // Or
spark.sql("SELECT * FROM us_origin_airport_JFK_tmp_view")

### 뷰 드롭

In [ ]:
# # -- In SQL
# DROP VIEW IF EXISTS us_origin_airport_SFO_global_tmp_view; 
# DROP VIEW IF EXISTS us_origin_airport_JFK_tmp_view

# // In Scala/Python
spark.catalog.dropGlobalTempView("us_origin_airport_SFO_global_tmp_view")
spark.catalog.dropTempView("us_origin_airport_JFK_tmp_view")

### 메타테이터 보기 97p

In [ ]:
#  // In Scala/Python
spark.catalog.listDatabases()
spark.catalog.listTables()
spark.catalog.listColumns("us_delay_flights_tbl")

### 캐싱

In [ ]:
# # -- In SQL
# # 테이블에 캐싱하기
# CACHE [LAZY] TABLE <table-name> 
# UNCACHE TABLE <table-name>

### 테이블을 데이터 프레임으로 읽기 98p
### SQL을 사용하여 테이블을 쿼리하고 반환된 결과를 데이터프레임에 저장

In [ ]:
us_flights_df = spark.sql("SELECT * FROM us_delay_flights_tbl")
us_flights_df2 = spark.table("us_delay_flights_tbl")

### 데이터 프레임 및 SQL 테이블을 위한 데이터 소스
### DataFrameReader

In [ ]:
# DataFrameReader.format(args).option(“key”,”value”).schema(args).load()
# SparkSession 인스턴스를 통해서만 DataFrameReader에 엑세스 할 수 있다.

# SparkSession.read # 정적 데이터 소스
# SparkSession.readStream # 스트리밍 소스

### DataFrameWriter

In [ ]:
# DataFrameWriter.format(args)
# 	.option(args)
# 	.bucketBy(args)
# 	.partitionBy(args)
# 	.save(path)

# DataFrameWriter.format(args).option(args).bucketBy(args).partitionBy(args).save(path)

# 위에서 설명한 read와 같다.
# DataFrame.write 
# // or 
# DataFrame.writeStream

### 파케이 파일 읽어오기 102p
### 데이터프레임으로 읽기

In [ ]:
# In Python
file = """PySpark/data/databricks-datasets/learning-spark-v2/flights/summary-data/parquet/2010-summary.parquet/"""
df = spark.read.format("parquet").load(file)
df.show(10)

### Spark SQL 테이블로 읽기 103p

In [ ]:
# -- In SQL
spark.sql("""
    CREATE OR REPLACE TEMPORARY VIEW us_delay_flights_tbl 
    USING parquet 
    OPTIONS (
        path "PySpark/data/databricks-datasets/learning-spark-v2/flights/summary-data/parquet/2010-summary.parquet/" )
""")

# In Python
spark.sql("SELECT * FROM us_delay_flights_tbl").show(10)

# # DataFrame으로 테이블을 읽어옵니다.
# us_delay_flights_df = spark.read.format("parquet").option("path", "PySpark/data/databricks-datasets/learning-spark-v2/flights/summary-data/parquet/2010-summary.parquet/").load()

# # 임시 뷰를 생성합니다.
# us_delay_flights_df.createOrReplaceTempView("us_delay_flights_tbl")

### 데이터 프레임을 파케이 파일로 쓰기 104p

In [ ]:
# In Python
(df.write.format("parquet")
    .mode("overwrite")
    .option("compression", "snappyb")
    .save("/tmp/data/parquet/df_parquet"))

### 스파크 SQL 테이블에 데이터 프레임 쓰기

In [ ]:
# In Python
(df.write
    .mode("overwrite")
    .saveAsTable("us_delay_flights_tbl"))

### JSON 파일을 데이터 프레임으로 읽기 105p

In [ ]:
# In Python
file = "/Users/hyunjun/vscode/Spark//databricks-datasets/learning-spark-v2/flights/summary-data/json/*"
df = spark.read.format("json").load(file)
df.show(10)

### 스파크 SQL 테이블로 JSON 파일 읽기 106p

In [ ]:
# -- In SQL
spark.sql("""
    CREATE OR REPLACE TEMPORARY VIEW us_delay_flights_tbl USING json
    OPTIONS (
    path "PySpark/data/databricks-datasets/learning-spark-v2/flights/summary-data/json/*"
    )
""")

#// In Scala/Python
spark.sql("SELECT * FROM us_delay_flights_tbl").show(10)


### 데이터 프레임을 JSON 파일로 쓰기

In [ ]:
# In Python
(df.write.format("json")
    .mode("overwrite")
    .option("compression", "snappy")
    .save("/tmp/data/json/df_json"))

### CSV 파일을 데이터 프레임으로 읽기 108p

In [ ]:
# In Python
file = "PySpark/data/databricks-datasets/learning-spark-v2/flights/summary-data/csv/*"
schema = "DEST_COUNTRY_NAME STRING, ORIGIN_COUNTRY_NAME STRING, count INT"
df = (spark.read.format("csv")
    .option("header", "true")# Exit if any errors .option("nullValue", "")
    .schema(schema)# Replace any null data field with quotes 
    .option("mode", "FAILFAST")
    .load(file))

In [ ]:
# -- In SQL
spark.sql("""CREATE OR REPLACE TEMPORARY VIEW us_delay_flights_tbl USING csv
OPTIONS (
path "PySpark/data/databricks-datasets/learning-spark-v2/flights/summary-data/csv/*", header "true",
inferSchema "true",
mode "FAILFAST"
)""")

In [ ]:
# // In Scala/Python
spark.sql("SELECT * FROM us_delay_flights_tbl").show(10)

### 데이터 프레임을 CSV 파일로 쓰기 109p

In [ ]:
# In Python
df.write.format("csv").mode("overwrite").save("/tmp/data/csv/df_csv")

### 에이브로 110p
### 에이브로 파일을 데이터 프레임으로 읽기

In [ ]:
df = spark.read.format("avro").load("PySpark/data/databricks-datasets/learning-spark-v2/flights/summary-data/avro/*")
df.show(truncate=False)

### 에이브로 파일을 스파크 SQL 테이블로 읽기 111p

In [ ]:
# -- In SQL
spark.sql("""CREATE OR REPLACE TEMPORARY VIEW episode_tbl USING avro
OPTIONS (
path "PySpark/data/databricks-datasets/learning-spark-v2/flights/summary-data/avro/*"
)""")

### 테이블 생성 후 SQL로 데이터 프레임 읽기

In [ ]:
# In Python
spark.sql("SELECT * FROM episode_tbl").show(truncate=False)

### 데이터 프레임을 에이브로 파일로 쓰기 112p

In [ ]:
# In Python
(df.write
  .format("avro")
  .mode("overwrite")
  .save("/tmp/data/avro/df_avro"))

### ORC 파일을 데이터 프레임으로 읽기 113p

In [ ]:
# In Python
file = "PySpark/data/databricks-datasets/learning-spark-v2/flights/summary-data/orc/*"
df = spark.read.format("orc").option("path", file).load()
df.show(10, False)

### 스파크 SQL 테이블로 ORC 파일 읽기 114p

In [ ]:
# -- In SQL
spark.sql(
    """CREATE OR REPLACE TEMPORARY VIEW us_delay_flights_tbl 
    USING orc
    OPTIONS (
    path "PySpark/data/databricks-datasets/learning-spark-v2/flights/summary-data/orc/*"
    )"""
)

In [ ]:
# // In Scala/Python
spark.sql("SELECT * FROM us_delay_flights_tbl").show(10)

### 데이터 프레임을 ORC 파일로 쓰기 

In [ ]:
# In Python
(df.write.format("orc")
    .mode("overwrite")
    .option("compression", "snappy")
    .save("/tmp/data/orc/flights_orc"))

### 이미지 파일을 데이터 프레임으로 읽기 116p

In [ ]:
# In Python
from pyspark.ml import image
image_dir = "PySpark/data/databricks-datasets/learning-spark-v2/cctvVideos/train_images/"
images_df = spark.read.format("image").load(image_dir)
images_df.printSchema()

In [ ]:
images_df.select("image.height", "image.width", "image.nChannels", "image.mode",
      "label").show(5, truncate=False)

### 데이터 프레임으로 이진 파일 읽기

In [ ]:
# In Python
path = "PySpark/data/databricks-datasets/learning-spark-v2/cctvVideos/train_images/"
binary_files_df = (spark.read.format("binaryFile")
    .option("pathGlobFilter", "*.jpg")
    .load(path))
binary_files_df.show(5)

### 파티션 데이터 검색 무시

In [ ]:
# In Python
binary_files_df = (spark.read.format("binaryFile")
    .option("pathGlobFilter", "*.jpg")
    .option("recursiveFileLookup", "true")
    .load(path))
binary_files_df.show(5)